In [18]:
import re,yaml,codecs
from os.path import expanduser
import pandas as pd
from cellbell import ding

In [19]:
home = expanduser("~")
# repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
serie=home+"/Github/Kalaba/Kalaba2/Kanonik/"
serie=home+"/pCloud Drive/FOD/GB/Copy/Recherche/Agathe/"

In [20]:
nomLexique=home+"/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
dfLexique=pd.read_csv(nomLexique,encoding="utf8",sep=";")

In [21]:
dfLexique.columns="ortho phono liaison cat ms tam lemme L23 F1 F2 F3 F4".split()
catLexique={"NOM":"N G".split(),"VER":"V".split(),"ADJ":"F G J K L M".split()}
tam="pi ii ai".split()
pers="3S 3P".split()

In [22]:
#dfLexique#[dfLexique["lemme"]=="coyote"]["ortho"].tolist()

In [23]:
newStems={}
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.safe_load(stream)
stems

{'ADJ': {'NuteS': ['en'],
  'TitiZ': ['court'],
  'Tonud': ['verte', 'vert', 'verts'],
  'badiN': ['mort'],
  'bewep': ['disparus'],
  'bubot': [u'pr\xe9sente'],
  'bunig': ['autres'],
  'gepaT': [u'm\xe9chants', u'm\xe9chantes', u'm\xe9chante'],
  'liboro': ['enfants', 'enfant'],
  'padol': [u'pr\xe8s'],
  'runuk': ['noire', 'noires', 'noir', 'noirs'],
  'ruwor': ['derniers'],
  'saDom': ['parents'],
  'savat': ['grande', 'grand'],
  'zuwoku': ['chat']},
 'DET': {'DoSif': ['par'],
  'Nepum': ['une'],
  'SaZaZ': ['ses'],
  'Telig': ['entre'],
  'berok': ['sur'],
  'bozapa': ['des'],
  'daDoT': ['au'],
  'dakog': ['six'],
  'damid': [u'troisi\xe8me'],
  'durib': [u'\xe0'],
  'firof': ['autres'],
  'jeTud': ['de'],
  'jiDes': ['les'],
  'kuZir': ['en'],
  'kumel': ['aux'],
  'kused': ['pendant'],
  'lerok': ['un'],
  'minuT': ['deux'],
  'nagud': ['quatre'],
  'nigib': ['sept'],
  'nonan': ['trois'],
  'nubola': ['avec'],
  'razop': ['dans'],
  'retod': ['du'],
  'riSaD': ['le'],
  'siva

In [47]:
def getFormesVerbe(verbe):
    p3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    p3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="pi")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    i3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    i3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ii")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    a3S=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3S"), "ortho"].tolist()[0]
    a3P=dfLexique.loc[dfLexique["cat"].isin(catLexique["VER"]) 
                          & (dfLexique["lemme"]==verbe)
                          & (dfLexique["tam"]=="ai")
                          & (dfLexique["ms"]=="3P"), "ortho"].tolist()[0]
    
    return p3S,p3P,i3S,i3P,a3S,a3P
                             
def getLemme(forme,cat):
    if cat not in catLexique:
        return
    else:
        lemmes=dfLexique[(dfLexique["ortho"]==forme)&(dfLexique["cat"].isin(catLexique[cat]))]["lemme"].tolist()
        if lemmes:
            return lemmes[0]
        else:
            return

def getFormes(forme,cat):
    print forme,cat
    lemme=getLemme(forme,cat)
    if lemme:
        if cat=="VER":
            flexion=list(getFormesVerbe(lemme))
        elif cat=="DET":
            if lemme=="DEF": flexion="le la les l".split()
            elif lemme=="IND": flexion="un une des dE d".split()
            else: flexion="ce cet cette ces".split()
        else:
            flexion=dfLexique.loc[dfLexique["cat"].isin(catLexique[cat]) & (dfLexique["lemme"]==lemme), "ortho"].tolist()
            flexion=list(set(flexion))
            if cat=="NOM" and lemme.endswith("s"):
                print "attention",lemme,"a été modifié avec un S"
                lemme=lemme[:-1]+"S"
            if lemme in flexion:
                flexion.remove(lemme)
        return [lemme]+flexion
    else:
        return

In [43]:
apiSymboles=u"ʃʒŋθð"
sampaSymboles=u"SZNTD"
translit={}
for i,s in enumerate(apiSymboles):
    translit[s]=sampaSymboles[i]
translit

{u'\xf0': u'D',
 u'\u014b': u'N',
 u'\u0283': u'S',
 u'\u0292': u'Z',
 u'\u03b8': u'T'}

In [44]:
def getTranslit(chaine):
    result=chaine
    for s in translit:
        result=result.replace(s,translit[s])
    return result

In [45]:
getTranslit(u"ʃaʒuŋoθið")

u'SaZuNoTiD'

In [48]:
for cat in stems:
    if cat not in newStems:
        newStems[cat]={}
    if cat not in ["PREP"]:
        print cat
        for element in stems[cat]:
            if isinstance(stems[cat][element],dict):
                newStems[cat][element]={}
                for word in stems[cat][element]:
                    if isinstance(stems[cat][element][word],dict):
                        subelement=word
                        if subelement not in newStems[cat][element]:
                            newStems[cat][element][subelement]={}
                        for w in stems[cat][element][subelement]:
                            mot=getTranslit(w)
                            flexion=stems[cat][element][subelement][w]
                            lemme=flexion[0]
                            print getFormes(lemme,cat)
                            newStems[cat][element][subelement][mot]=getFormes(lemme,cat)                            
                    else:
                        mot=getTranslit(word)
                        flexion=stems[cat][element][word]
                        lemme=flexion[0]
                        print lemme, cat
                        print getFormes(lemme,cat)
                        newStems[cat][element][mot]=getFormes(lemme,cat)
                    
            else:
                flexion=stems[cat][element]
                lemme=flexion[0]
                print lemme
                print getFormes(lemme,cat)
                mot=getTranslit(element)
                newStems[cat][mot]=getFormes(lemme,cat)        
    else:
        print cat
        for element in stems[cat]:
            mot=getTranslit(element)
            newStems[cat][mot]=stems[cat][element]

ADJ
grande
grande ADJ
[u'grand', u'grands', u'grande', u'grandes']
grande ADJ
mort
mort ADJ
[u'mort', u'morts', u'mortes', u'morte']
mort ADJ
autres
autres ADJ
[u'autre', u'autres']
autres ADJ
court
court ADJ
[u'court', u'courte', u'courtes', u'courts']
court ADJ
disparus
disparus ADJ
[u'dispara\xeetre', u'disparues', u'disparue', u'disparu', u'disparus']
disparus ADJ
parents
parents ADJ
[u'parent', u'parentes', u'parente', u'parents']
parents ADJ
présente
présente ADJ
[u'pr\xe9sent', u'pr\xe9sentes', u'pr\xe9sents', u'pr\xe9sente']
présente ADJ
en
en ADJ
None
en ADJ
noire
noire ADJ
[u'noir', u'noire', u'noires', u'noirs']
noire ADJ
verte
verte ADJ
[u'vert', u'verte', u'vertes', u'verts']
verte ADJ
méchants
méchants ADJ
[u'm\xe9chant', u'm\xe9chante', u'm\xe9chantes', u'm\xe9chants']
méchants ADJ
chat
chat ADJ
[u'chat', u'chats', u'chatte', u'chattes']
chat ADJ
près
près ADJ
None
près ADJ
derniers
derniers ADJ
[u'dernier', u'derni\xe8re', u'derniers', u'derni\xe8res']
derniers ADJ
enfa

[u'voir', u'voit', u'voient', u'voyait', u'voyaient', u'vit', u'virent']
voient VER
coururent
coururent VER
[u'courir', u'court', u'courent', u'courait', u'couraient', u'courut', u'coururent']
coururent VER
retournent
retournent VER
[u'retourner', u'retourne', u'retournent', u'retournait', u'retournaient', u'retourna', u'retourn\xe8rent']
retournent VER
lever
lever VER
[u'lever', u'l\xe8ve', u'l\xe8vent', u'levait', u'levaient', u'leva', u'lev\xe8rent']
lever VER
excuses
excuses VER
[u'excuser', u'excuse', u'excusent', u'excusait', u'excusaient', u'excusa', u'excus\xe8rent']
excuses VER
enlevaient
enlevaient VER
[u'enlever', u'enl\xe8ve', u'enl\xe8vent', u'enlevait', u'enlevaient', u'enleva', u'enlev\xe8rent']
enlevaient VER
ressuscite
ressuscite VER
[u'ressusciter', u'ressuscite', u'ressuscitent', u'ressuscitait', u'ressuscitaient', u'ressuscita', u'ressuscit\xe8rent']
ressuscite VER
vote
vote VER
[u'voter', u'vote', u'votent', u'votait', u'votaient', u'vota', u'vot\xe8rent']
vote VER

[u'six']
six NOM
mère
mère NOM
None
mère NOM


In [49]:
newStems

{'ADJ': {u'NuteS': None,
  u'TitiZ': [u'court', u'courte', u'courtes', u'courts'],
  u'Tonud': [u'vert', u'verte', u'vertes', u'verts'],
  u'badiN': [u'mort', u'morts', u'mortes', u'morte'],
  u'bewep': [u'dispara\xeetre',
   u'disparues',
   u'disparue',
   u'disparu',
   u'disparus'],
  u'bubot': [u'pr\xe9sent', u'pr\xe9sentes', u'pr\xe9sents', u'pr\xe9sente'],
  u'bunig': [u'autre', u'autres'],
  u'gepaT': [u'm\xe9chant', u'm\xe9chante', u'm\xe9chantes', u'm\xe9chants'],
  u'liboro': [u'enfant', u'enfants'],
  u'padol': None,
  u'runuk': [u'noir', u'noire', u'noires', u'noirs'],
  u'ruwor': [u'dernier', u'derni\xe8re', u'derniers', u'derni\xe8res'],
  u'saDom': [u'parent', u'parentes', u'parente', u'parents'],
  u'savat': [u'grand', u'grands', u'grande', u'grandes'],
  u'zuwoku': [u'chat', u'chats', u'chatte', u'chattes']},
 'DET': {u'DoSif': None,
  u'Nepum': None,
  u'SaZaZ': None,
  u'Telig': None,
  u'berok': None,
  u'bozapa': None,
  u'daDoT': None,
  u'dakog': None,
  u'damid

In [50]:
with open(serie+"newStems.yaml", 'w') as stream:
    yaml.safe_dump(newStems,stream=stream,default_flow_style=False,allow_unicode=True)

In [52]:
with open(repertoire+"newStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [53]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  NuteS: [null]
  TitiZ: [court, courte, courtes, courts]
  Tonud: [vert, verte, vertes, verts]
  badiN: [mort, morts, mortes, morte]
  bewep: [disparaître, disparues, disparue, disparu, disparus]
  bubot: [présent, présentes, présents, présente]
  bunig: [autre, autres]
  gepaT: [méchant, méchante, méchantes, méchants]
  liboro: [enfant, enfants]
  padol: [null]
  runuk: [noir, noire, noires, noirs]
  ruwor: [dernier, dernière, derniers, dernières]
  saDom: [parent, parentes, parente, parents]
  savat: [grand, grands, grande, grandes]
  zuwoku: [chat, chats, chatte, chattes]
DET:
  DoSif: [null]
  Nepum: [null]
  SaZaZ: [null]
  Telig: [null]
  berok: [null]
  bozapa: [null]
  daDoT: [null]
  dakog: [null]
  damid: [null]
  durib: [null]
  firof: [null]
  jeTud: [null]
  jiDes: [null]
  kuZir: [null]
  kumel: [null]
  kused: [null]
  lerok: [null]
  minuT: [null]
  nagud: [null]
  nigib: [null]
  nonan: [null]
  nubola: [null]
  razop: [null]
  retod: [null]
  riSaD: [null]
  siv

In [54]:
with open(repertoire+'newStems.yaml', 'w') as output:
    output.write(yamlText)

In [51]:
ding()

ImportError: Library "c" not found.